# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Данные

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import dump

df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


### Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

- *сalls* — количество звонков,
- *minutes* — суммарная длительность звонков в минутах,
- *messages* — количество sms-сообщений,
- *mb_used* — израсходованный интернет-трафик в Мб,
- *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


С данными все ок, пропусков нет. Можно приступать к исследованию.

## 2. Разбивка на выборки

In [4]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

In [5]:
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

Данные разбиты на обучающую, валидационную и тестовые выборки в соотношении 60% : 20% : 20%.

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

## 3. Исследование моделей

In [7]:
#Исследуем модель "Дерево решений"
for depth in range(2, 23, 2):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree_valid = model_tree.predict(features_valid)
    accuracy_tree_valid = accuracy_score(target_valid, predictions_tree_valid)
    print(accuracy_tree_valid, '||', depth)

0.7822706065318819 || 2
0.7791601866251944 || 4
0.7838258164852255 || 6
0.7791601866251944 || 8
0.7744945567651633 || 10
0.7620528771384136 || 12
0.7589424572317263 || 14
0.7340590979782271 || 16
0.7309486780715396 || 18
0.7216174183514774 || 20
0.7262830482115086 || 22


0.78 - неплохой результат. Проверим остальные модели.

In [8]:
#модель случайного леса
for estim in range(4, 73, 4):
    model_forest = RandomForestClassifier(n_estimators=estim, random_state=12345)
    model_forest.fit(features_train, target_train)
    predictions_forest_valid = model_forest.predict(features_valid)
    accuracy_forest_valid = accuracy_score(target_valid, predictions_forest_valid)
    print(accuracy_forest_valid, '||', estim)

0.7713841368584758 || 4
0.7822706065318819 || 8
0.7869362363919129 || 12
0.7869362363919129 || 16
0.7869362363919129 || 20
0.7900466562986003 || 24
0.7838258164852255 || 28
0.7822706065318819 || 32
0.7807153965785381 || 36
0.7838258164852255 || 40
0.7900466562986003 || 44
0.7916018662519441 || 48
0.7900466562986003 || 52
0.7853810264385692 || 56
0.7853810264385692 || 60
0.7807153965785381 || 64
0.7807153965785381 || 68
0.7791601866251944 || 72


Наиболее точно на исследованном диапазоне RandomForest работает при гиперпараметре n_estimators = 24.

In [9]:
#Модель логистической регрессии
model_logistic = LogisticRegression(random_state=12345, solver='liblinear') 

In [10]:
model_logistic.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [11]:
predictions_logistic_valid = model_logistic.predict(features_valid)

In [12]:
accuracy_logistic_valid = accuracy_score(target_valid, predictions_logistic_valid)
accuracy_logistic_valid

0.7589424572317263

### Вывод

Дерево решений показало наименее точный результат (accuracy = 0.71). Accuracy логической регрессии (0.759) соответствует требованию задачи. Наилучший результат показала модель случайного леса, при n_estimators = 24 accuracy = 0.79, остановимся на ней.

## 4. Проверка модели на тестовой выборке

In [13]:
model_forest = RandomForestClassifier(n_estimators=24, random_state=12345)

In [14]:
model_forest.fit(features_train, target_train)
predictions = model_forest.predict(features_test)

In [15]:
accuracy = accuracy_score(target_test, predictions)
accuracy

0.7822706065318819

Точность больше 0.75, задача выполнена.

## 5. Проверка модели на адекватность

In [16]:
#Проверка на адекватность: представим, что у нас константные значения, равные единицы.
target_test.sum() / target_test.count()

0.3157076205287714

0.32 < 0.78 => выбранная модель адекватна.

Посмотрим на баланс классов в датасете.

In [17]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Видим, что около 69% клиентов пользуются тарифом «Смарт». Пусть тогда наша константная модель всегда предсказывает этот тариф (т.е. возвращает ноль). Тогда средняя точность модели будет равна 69%. Но точность нашей лучшей модели всё равно выше и равна примерно 78%, поэтому проверка на адекватность пройдена с успехом.